In [88]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [89]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_05 import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

In [90]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train,y_train),Dataset(x_valid,y_valid)
nh,bs = 50,512
c=y_train.max().item()+1
loss_func = F.cross_entropy

In [92]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [93]:
#export
class Callback():
  _order = 0
  def set_runner(self, run): self.run = run
  def __getattr__(self, k): return getattr(self.run, k)

  @property
  def name(self):
    name = re.sub(r'Callback$', '', self.__class__.__name__)
    return camel2snake(name or 'callback')

  def __call__(self, cb_name):
    f = getattr(self, cb_name, None)
    if f and f(): return True
    return False

class TrainEvalCallback(Callback):
  def begin_fit(self):
    self.run.n_epochs=0.
    self.run.n_iter=0
  
  def after_batch(self):
    if not self.in_train:return
    self.run.n_epochs += 1./self.iters
    self.run.n_iter += 1

  def begin_epoch(self):
    self.run.n_epochs = self.epoch
    self.model.train()
    self.run.in_train = True

  def begin_validate(self):
    self.model.eval()
    self.run.in_train=False

class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [94]:
#export
class Runner():
  def __init__(self, cbs=None,cb_funcs=None):
    cbs = listify(cbs)
    for cbf in listify(cb_funcs):
      cb = cbf()
      setattr(self, cb.name, cb)
      cbs.append(cb)
    self.stop,self.cbs, self.in_train = False, [TrainEvalCallback()]+cbs, True
    #added self.in_train to get code working
  @property
  def opt(self):        return self.learn.opt
  @property
  def model(self):      return self.learn.model
  @property
  def loss_func(self):  return self.learn.loss_func
  @property
  def data(self):       return self.learn.data

  def one_batch(self, xb, yb):
    try:
      self.xb,self.yb = xb, yb
      self('begin_batch')
      self.pred = self.model(self.xb)
      self('after_pred')
      self.loss = self.loss_func(self.pred, self.yb)
      self('after_loss')
      if not self.in_train: return
      self.loss.backward()
      self('after_backward')
      self.opt.step()
      self('after_step')
      self.opt.zero_grad()
    except CancelBatchException: self('after_cancel_batch')
    finally: self('after_batch')
  
  def all_batches(self, dl):
    self.iters = len(dl)
    try:
      for xb, yb in dl: self.one_batch(xb,yb)
    except CancelEpochException: self('after_cancel_epoch')

  def fit(self, epochs, learn):
    self.epochs, self.learn, self.loss = epochs,learn,tensor(0.)

    try:
      for cb in self.cbs: cb.set_runner(self)
      self('begin_fit')
      for epoch in range(epochs):
        self.epoch = epoch
        if not self('begin_fit'): self.all_batches(self.data.train_dl)

        with torch.no_grad():
          if not self('begin_validate'): self.all_batches(self.data.valid_dl)
        self('after_epoch')

    except CancelTrainException: self('after_cancel_train')
    finally:
      self('after_fit')
      self.learn = None

  def __call__(self, cb_name):
    res = False
    for cb in sorted(self.cbs, key=lambda x: x._order):res = cb(cb_name) and res
    return res

In [95]:
learn = create_learner(get_model, loss_func, data)

In [96]:
class TestCallback(Callback):
  _order = 1
  def after_step(self):
    print(self.n_iter)
    if self.n_iter>=10: raise CancelTrainException

In [97]:
run = Runner(cb_funcs = TestCallback)

In [98]:
run.fit(3, learn)

0
1
2
3
4
5
6
7
8
9
10


**Other Callbacks**

In [99]:
class LR_Find(Callback):
  _order = 1
  def __init__(self, max_iter=100, min_lr=1e-6,max_lr=10):
    self.max_iter, self.min_lr,self.max_lr = max_iter,min_lr,max_lr
    self.best_loss =1e9

  def begin_batch(self):
    if not self.in_train: return
    pos = self.n_iter/self.max_iter
    lr = self.min_lr * (self.max_lr/self.min_lr) ** pos
    for pg in self.opt.param_groups: pg['lr'] = lr
  
  def after_step(self):
    if self.n_iter>=self.max_iter or self.loss>self.best_loss*10:
      raise CancelTrainException()
    if self.loss < self.best_loss: self.best_loss = self.loss

In [100]:
learn = create_learner(get_model, loss_func, data)

In [101]:
run = Runner(cb_funcs=[LR_Find, Recorder])

In [102]:
run.fit(2,learn)

TypeError: ignored

In [103]:
run.recorder.plot(skip_last=5)

AttributeError: ignored

In [104]:
!pip install fire

In [105]:
!python notebook2script.py 05b_early_stopping.ipynb

Converted 05b_early_stopping.ipynb to exp/nb_05b.py
